In [178]:
# conda install -c conda-forge jellyfish
import jellyfish._jellyfish as pyjellyfish
from eudex import eudex
from transformers import BertTokenizer, BertModel, pipeline
import pandas as pd
import openpyxl

In [185]:
jf = pyjellyfish
e = eudex
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [398]:
w1 = 'loose'
w2 = 'lose'

In [399]:
jf.soundex(w1), jf.soundex(w2)

('L200', 'L200')

In [400]:
e(w1), e(w2)

(5764607523034240000, 5764607523034240000)

In [401]:
soundexDistance(w1,w2)

0

In [402]:
eudexDistance(w1,w2)

0

In [344]:
def eudexDistance(w1,w2):
    result = sum(1 for _ in bin(e(w1) ^ e(w2)) if _ == '1') # number of one after xoring hashes
    return result

In [345]:
def soundexDistance(w1,w2):
    temp1 = sum (1 for _ in bin(ord(jf.soundex(w1)[0]) ^ ord(jf.soundex(w2)[0])) if _ == '1')
    temp2 = sum (1 for _ in bin(int(jf.soundex(w1)[1:]) ^ int(jf.soundex(w2)[1:])) if _ == '1')
    
    result = temp1 + temp2
    return result

In [346]:
def getDistanceList(s1, s2):
    M = [[0] * (len(s1)+1) for _ in range(len(s1)+1)]# 길이가 긴 문자열을 기준으로 2차원 리스트 초기
    long = len(s1)
    short = len(s2)        
    for i in range(long+1):
        M[i][0] = i
    for j in range(short+1):
        M[0][j] = j
    for i in range(1, long+1):
        for j in range(1, short+1):
            if s1[i-1] == s2[j-1]:
                M[i][j] = M[i-1][j-1]
            else:
                M[i][j] = min(M[i-1][j], M[i-1][j-1], M[i][j-1]) + 1
    return M, M[long][short] 

In [347]:
# 두 문자열 길이비교
def compareLength(s1, s2):
    if len(s1) >= len(s2):
        longstr, shortstr = s1, s2
    else:
        longstr, shortstr = s2, s1
    return longstr, shortstr


In [323]:
text1 = "Could you find me my sit, please?" # seat

In [348]:
text1 = "They are with their docs." # their

In [374]:
text1 = "We walk along the saend." # their

In [390]:
text1 = "She ramed the key into the lock." # their

In [403]:
text1 = "How can they loose the game."

In [408]:
text_split = list(map(str,text1.lower().split()))
text_list = []
for i in range(len(text_split)):
    temp = list(text_split)
    temp[i] = '[MASK]'
    if i == len(text_split)-1:
        if text1[-1] == '?':
            temp[i] = '[MASK]?'
        else:
            temp[i] = '[MASK].'
    text_list.append(' '.join(map(str,temp)))
    print(' '.join(map(str,temp)))
text_split

[MASK] can they loose the game.
how [MASK] they loose the game.
how can [MASK] loose the game.
how can they [MASK] the game.
how can they loose [MASK] game.
how can they loose the [MASK].


['how', 'can', 'they', 'loose', 'the', 'game.']

In [411]:
fill2 = []
data3 = []
for num in range(len(text_split)):
    masked_sentence = text_list[num]
    original_token = text_split[num]
    print(masked_sentence, " original token:", original_token)
    fill2.append(unmasker(masked_sentence, original_token = original_token, top_k=30000))
    for i in range(len(fill2)):
        for j in range(len(fill2[i])):
            s1 = fill2[i][j].get('token_str')
            s2 = original_token
            s1, s2 = compareLength(s1, s2) #문자열의 길이를 비교해서 s1에 긴문자열, s2에 짧은 문자열을 넣는다.
    
            soundex_distance = soundexDistance(s1, s2)
            M, edit_distance = getDistanceList(s1, s2)
            fill2[i][j]['edit_distance'] = edit_distance
            #print(s2, "와(과)", s1, "의 최소 편집 거리는", edit_distance , "이다.", end="\n\n")
            fill2[i][j]['soundex_distance'] = soundex_distance
    result3 = sorted(fill2[num], key = lambda x:(x['soundex_distance'],-x['score'],x['edit_distance']))
    print('token   sequence                       soundex  edit_dist score')   
    for k in range(4):
        print('%-5s'%result3[k].get('token_str'),end='\t')
        print('%-28s'%result3[k].get('sequence'),end='\t')
        print('%-7s'%result3[k].get('soundex_distance'),end='\t')
        print('%-6s'%result3[k].get('edit_distance'),end='\t')
        print('%-5s'%result3[k].get('score'))
    print()

[MASK] can they loose the game.  original token: how
token   sequence                       soundex  edit_dist score
how  	how can they loose the game.	0      	0     	0.20777511596679688
he   	he can they loose the game. 	0      	2     	0.0013188411248847842
hey  	hey can they loose the game.	0      	2     	1.9575669284677133e-05
ho   	ho can they loose the game. 	0      	1     	1.1874297342728823e-05

how [MASK] they loose the game.  original token: can
token   sequence                       soundex  edit_dist score
can  	how can they loose the game.	0      	0     	0.03041735105216503
come 	how come they loose the game.	0      	3     	0.001543902326375246
came 	how came they loose the game.	0      	2     	0.0003435284597799182
cain 	how cain they loose the game.	0      	1     	9.032014531840105e-06

how can [MASK] loose the game.  original token: they
token   sequence                       soundex  edit_dist score
they 	how can they loose the game.	0      	0     	0.06572222709655762
t

In [410]:
fill2 = []
result4=[]
for num in range(len(text_split)):
    masked_sentence = text_list[num]
    original_token = text_split[num]
    print(masked_sentence, " original token:", original_token)
    fill2.append(unmasker(masked_sentence, original_token = original_token, top_k=30000))
    for i in range(len(fill2)):
        for j in range(len(fill2[i])):
            s1 = fill2[i][j].get('token_str')
            s2 = original_token
            s1, s2 = compareLength(s1, s2) #문자열의 길이를 비교해서 s1에 긴문자열, s2에 짧은 문자열을 넣는다.
    
            eudex_distance = eudexDistance(s1, s2)
            soundex_distance = soundexDistance(s1, s2)        
            M, edit_distance = getDistanceList(s1, s2)
            fill2[i][j]['edit_distance'] = edit_distance
            fill2[i][j]['eudex_distance'] = eudex_distance
            fill2[i][j]['soundex_distance'] = soundex_distance
    result4 = sorted(fill2[num], key = lambda x:(x['soundex_distance'],x['edit_distance'],-x['score']))
    print('token   sequence                      soundex  eudex  edit_dist  score')   
    for k in range(4):
        print('%-5s'%result4[k].get('token_str'),end='\t')
        print('%-28s'%result4[k].get('sequence'),end='\t')
        print('%-7s'%result4[k].get('soundex_distance'),end='\t')
        print('%-7s'%result4[k].get('eudex_distance'),end='\t')       
        print('%-7s'%result4[k].get('edit_distance'),end='\t')
        print('%-5s'%result4[k].get('score'))
    print()

[MASK] can they loose the game.  original token: how
token   sequence                            soundex  eudex  edit_dist  score
how  	how can they loose the game.	0      	0      	0      	0.20777511596679688
ho   	ho can they loose the game. 	0      	0      	1      	1.1874297342728823e-05
howe 	howe can they loose the game.	0      	0      	1      	2.0198568506657466e-07
he   	he can they loose the game. 	0      	0      	2      	0.0013188411248847842

how [MASK] they loose the game.  original token: can
token   sequence                            soundex  eudex  edit_dist  score
can  	how can they loose the game.	0      	0      	0      	0.03041735105216503
cain 	how cain they loose the game.	0      	0      	1      	9.032014531840105e-06
con  	how con they loose the game.	0      	0      	1      	5.7095112424576655e-06
cam  	how cam they loose the game.	0      	1      	1      	1.1915569757547928e-06

how can [MASK] loose the game.  original token: they
token   sequence                   

In [378]:
df1 = pd.DataFrame(data1, columns=['token_str','sequence','score','sound_distance'])
df2 = pd.DataFrame(data2, columns=['token_str','sequence','sound_distance','edit_distance','score'])
df3 = pd.DataFrame(data3, columns=['token_str','sequence','sound_distance','edit_distance','score'])

In [133]:
print(df1.head())

  token_str                           sequence     score sound_distance
0       can    can you find me my sit, please?  0.544499           None
1      will   will you find me my sit, please?  0.198092           None
2     could  could you find me my sit, please?  0.151333           None
3     would  would you find me my sit, please?  0.087990           None
4       you  could you find me my sit, please?  0.954546           None


In [134]:
with pd.ExcelWriter('/home/tako/youngmi/errordetection/seat_result.xlsx') as writer:
    df1.to_excel(writer, index=False, sheet_name='sheet1')
    df2.to_excel(writer, index=False, sheet_name='sheet2')
    df3.to_excel(writer, index=False, sheet_name='sheet3')